# Cgauss in PyTorch --- check randome start values
Correlated gaussian basis functions

In [1]:
import numpy as np
import torch as th

import time
import cProfile

In [2]:
dtype = th.float64

gpuid = 0
#device = th.device("cuda:"+ str(gpuid))
device = th.device("cpu")

print("Execution device: ",device)
print("PyTorch version: ", th.__version__ )
print("CUDA available: ", th.cuda.is_available())
print("CUDA version: ", th.version.cuda)
print("CUDA device:", th.cuda.get_device_name(gpuid))

Execution device:  cpu
PyTorch version:  0.4.1
CUDA available:  True
CUDA version:  9.0.176
CUDA device: TITAN V


In [3]:
# Utility functions
def vech2L(v,n):
    count = 0
    L = th.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            L[i,j]=v[count]
            count = count + 1
    return th.tensor(L , device=device, dtype=dtype)

# batched vech2L input is "X" as V nb x n(n+1)/2
def bvech2L(V,nb,n):
    count = 0
    L = th.zeros((nb,n,n))
    for j in range(n):
        for i in range(j,n):
            L[...,i,j]=V[...,count]
            count = count + 1
    return th.tensor(L , device=device, dtype=dtype)

# Batched Cholesky decomp
def cholesky(A):
    L = th.zeros_like(A)
    
    for i in range(A.shape[-1]):
        for j in range(i+1):
            s = 0.0
            for k in range(j):
                s = s + L[...,i,k].clone() * L[...,j,k].clone()
            
            L[...,i,j] = th.sqrt(A[...,i,i] - s) if (i == j) else \
                      (1.0 / L[...,j,j].clone() * (A[...,i,j] - s))
    return L

# Batched inverse of lower triangular matrices 
def inverseL(L):
    n = L.shape[-1]
    invL = th.zeros_like(L)
    for j in range(0,n):
        invL[...,j,j] = 1.0/L[...,j,j]
        for i in range(j+1,n):
            S = 0.0
            for k in range(i+1):
                S = S - L[...,i,k]*invL[...,k,j].clone()
            invL[...,i,j] = S/L[...,i,i]

    return invL
            

In [123]:
def b_energyeig(x,n,nb,Mass,Qmat,Sym,symc):
    
    nx = len(x);
    nn = int(n*(n+1)/2);
    nsym = len(symc);
    
    # extract linear coefs "eigen vector"
    c = x[-nb:];
    # reshape non-linear variables for easier indexing
    X = th.reshape(x[:nb*nn], (nb,nn))
    
    # generate tensor of lower triangular matrices from X
    # these are the non-linear parameters of the basis set
    L = th.zeros((nb,n,n), device=device, dtype=dtype)
    L = bvech2L(X,nb,n)
    
    # get the determinates for L |L| is the product of diag elements
    detL = th.abs(th.prod(th.diagonal(L, offset=0, dim1=-1, dim2=-2),1))
    
    # create the tensor of matrix products of the L matrices AKL = L x Ltranspose
    AK = th.matmul(L,th.transpose(L, 1, 2))

    
    # Initialize H T V and S matrices
    # H = T + V, we are solving (H-ES)c = 0 for E (energy)
    H = th.zeros((nb,nb), device=device, dtype=dtype);
    S = th.zeros((nb,nb), device=device, dtype=dtype);
    T = th.zeros((nb,nb), device=device, dtype=dtype);
    V = th.zeros((nb,nb), device=device, dtype=dtype);
    

    # outer loop is over symmetry terms, the matrices are summed over these sym terms
    for k in range(0,nsym):
        
        P = Sym[k,:,:]
        # symetry projection is applied only to "ket" this constructs AL
        AL = th.matmul(th.t(P), th.matmul(AK,P))

        # Akl = Ak + Al
        AKL = th.zeros((nb,nb,n,n), device=device, dtype=dtype)
        #for i in range(nb):
        #    for j in range(nb):
        #        #AKL[i,j] =  th.add(AK[i], AL[j])
        #        AKL[i,j] =  AK[i] + AL[j]
        AKL = AL.repeat((nb,1,1,1)) + th.transpose(AK.repeat((nb,1,1,1)), 0,1)
        
        # get the Cholesky decomp of all Akl martices
        cholAKL = cholesky(AKL)
        
        # get determinates of AKL from diags |Akl|= |Lk|**2
        detAKL = th.prod(th.diagonal(cholAKL, offset=0, dim1=-1, dim2=-2),-1)**2
        
        # compute inverses of lower tringular matrices in cholAKL
        invLKL = inverseL(cholAKL)
        
        # inverses Akl^-1 = Lkl' x Lkl
        invAKL = th.matmul(th.transpose(invLKL, dim0=-1, dim1=-2),invLKL)

        # get terms needed for potential energy V
        RIJ = th.zeros_like(invAKL, device=device, dtype=dtype);
        # 1/rij i~=j
        for j in range(0,n-1):
            for i in range(j+1,n):
                tmp2 = invAKL[...,i,i] + invAKL[...,j,j] - 2*invAKL[...,i,j];
                RIJ[...,i,j] = th.rsqrt(tmp2)

        # 1/rij i=j
        for i in range(0,n):
            RIJ[...,i,i] = th.rsqrt(invAKL[...,i,i])    

        # MATRIX ELEMENTS
        
        # Overlap: (normalized)
        # Skl = 2^3n/2 (||Lk|| ||Ll||/|AKL|)^3/2
        SKL = 2**(n*1.5) * th.sqrt( th.pow(th.ger(detL, detL)/detAKL ,3) );

        # Kinetic energy
        #TKL = SKL*(6*th.trace(Mass@Ak@invAkl@Al)) = skl*(6*th.sum(Mass*(Ak@invAkl@Al)))

        Tmat = th.zeros_like(invAKL)
        #for i in range(nb):
        #    for j in range(nb):
        #        Tmat[i,j] = (AK[i]@invAKL[i,j]@AL[j])
        Tmat = th.matmul(th.transpose(AK.repeat((nb,1,1,1)), 0,1), th.matmul(invAKL,AL))
        TKL = 6*SKL*th.sum(Mass*Tmat, dim=(-2,-1))

        # potential energy
        TWOoSqrtPI = 1.1283791670955126 # 2/sqrt(pi)
        
        VKL = TWOoSqrtPI*SKL*th.sum(RIJ*Qmat, dim=(-2,-1))
    
        # accumulate matrices over sym terms
        S = S + symc[k]*SKL
        T = T + symc[k]*TKL
        V = V + symc[k]*VKL
        
    # Hamiltonian
    H = T + V
    
    # complete lower triangle of H and S
    for i in range(0,nb):
        for j in range(i+1,nb):
            H[j,i] = H[i,j]
            S[j,i] = S[i,j]
            #H[i,j] = H[j,i];
            #S[i,j] = S[j,i];

    # compute Raylie quotent (it is the smallest energy eigen value when minimized over c)
    cHc = c@H@c;
    cSc = c@S@c;
    eng = cHc/cSc;
    
    #eng, _ = th.eig(th.matmul(th.inverse(S),H), eigenvectors=False) 
    
    return eng
    #return eng[0,0]           

In [218]:
def opt_energyrc():
    
    #
    # Li BO setup
    #
    n=3;
    
    Mass = th.tensor([[0.5, 0.0, 0.0],
                     [0.0, 0.5, 0.0],
                     [0.0, 0.0, 0.5]], device=device, dtype=dtype);
    
    Charge = th.tensor([-3, 1, 1, -3, 1, -3], device=device, dtype=dtype);
    Charge = vech2L(Charge,n)
    
    # symmetry projection terms
    Sym = th.zeros((6,3,3), device=device, dtype=dtype)
    # (1)(2)(3)
    Sym[0,:,:] = th.tensor([[1,0,0],[0,1,0],[0,0,1]], device=device, dtype=dtype);
    # (12)
    Sym[1,:,:] = th.tensor([[0,1,0],[1,0,0],[0,0,1]], device=device, dtype=dtype);
    # (13)
    Sym[2,:,:] = th.tensor([[0,0,1],[0,1,0],[1,0,0]], device=device, dtype=dtype);
    # (23)
    Sym[3,:,:] = th.tensor([[1,0,0],[0,0,1],[0,1,0]], device=device, dtype=dtype);
    # (123)
    Sym[4,:,:] = th.tensor([[0,1,0],[0,0,1],[1,0,0]], device=device, dtype=dtype);
    # (132)
    Sym[5,:,:] = th.tensor([[0,0,1],[1,0,0],[0,1,0]], device=device, dtype=dtype);

    # coeff's
    symc = th.tensor([4.0,4.0,-2.0,-2.0,-2.0,-2.0], device=device, dtype=dtype);

    # Sample parameters should return energy of -7.3615
    xvechL=th.tensor([
     1.6210e+00,
    -2.1504e-01,
     9.0755e-01,
     9.7866e-01,
    -2.8418e-01,
    -3.5286e+00,
    -3.3045e+00,
    -4.5036e+00,
    -3.2116e-01,
    -7.1901e-02,
     1.5167e+00,
    -8.4489e-01,
    -2.1377e-01,
    -3.6127e-03,
    -5.3774e-03,
    -2.1263e+00,
    -2.5191e-01,
     2.1235e+00,
    -2.1396e-01,
    -1.4084e-03,
    -1.0092e-02,
     4.5349e+00,
     9.4837e-03,
     1.1225e+00,
    -2.1315e-01,
     5.8451e-02,
    -4.9410e-03,
     5.0853e+00,
     7.3332e-01,
     5.0672e+00,
    -2.1589e-01,
    -6.8986e-03,
    -1.4310e-02,
     1.5979e+00,
     3.3946e-02,
    -8.7965e-01,
    -1.1121e+00,
    -2.1903e-03,
    -4.6925e-02,
     2.1457e-01,
     3.3045e-03,
     4.5120e+00,
    -2.1423e-01,
    -1.6493e-02,
    -2.3429e-03,
    -8.6715e-01,
    -6.7070e-02,
     1.5998e+00
     ], device=device, dtype=dtype, requires_grad=False)
    
    evec = th.tensor([
      -6.0460e-02,
       7.7708e-05,
       1.6152e+00,
       9.5443e-01,
       1.1771e-01,
       3.2196e+00,
       9.6344e-01,
       3.1398e+00
    ], device=device, dtype=dtype, requires_grad=False)
    
    # uncomment following lines to test above 
    #nb=8
    #x1 = th.tensor(th.cat((xvechL,evec)), device=device, dtype=dtype, requires_grad=True)
    #energy = b_energyeig(x1,n,nb,Mass,Charge,Sym,symc) 
    #print('energy {}'.format(energy)) # should be -7.3615
    #return x1
    
    # random start point
    nb=128;
    count = 1
    esum = 0.0
    th.manual_seed(3)
    #for i in range(count):
    #    x1 = th.empty(int(nb*n*(n+1)/2 + nb), device=device, dtype=dtype, requires_grad=True)
    #    th.nn.init.uniform_(x1, a=-.8, b=.8)
    #    #th.nn.init.normal_(x1, mean=0, std=.5)
    #    #x1 = th.randn(int(nb*n*(n+1)/2 + nb) , device=device, dtype=dtype, requires_grad=True)
    #    energy = b_energyeig(x1,n,nb,Mass,Charge,Sym,symc)
    #    esum += energy
    #    print('energy {}'.format(energy)) # should be -7.3615
    #print('Avg {}'.format(esum/count))
    #return x1
    
    # start from a restart value
    x1 = xrestart
    #print(energy)
    #return x1
    
    # Do the Optimization
    #optimizer = th.optim.LBFGS([x1])
    #optimizer = th.optim.Adadelta([x1], lr=160.0)
    #optimizer = th.optim.Adam([x1], lr=0.4)
    #optimizer = th.optim.Adagrad([x1], lr=1.0, lr_decay=0, weight_decay=0)
    #optimizer = th.optim.Adamax([x1], lr=0.2, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    #optimizer = th.optim.RMSprop([x1], lr=0.1, alpha=0.99, eps=1e-08, weight_decay=0, momentum=.1, centered=False)
    optimizer = th.optim.Rprop([x1], lr=0.0001, etas=(0.5, 1.2), step_sizes=(1e-06, 50))
    #optimizer = th.optim.SGD([x1], lr=1, momentum=0, dampening=0, weight_decay=0, nesterov=False)
    
    
    #scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer,threshold=0.00001,cooldown=3, verbose=True,patience=2, factor=0.5)
    
    for i in range(2000):
        optimizer.zero_grad()
        loss = b_energyeig(x1,n,nb,Mass,Charge,Sym,symc)
        loss.backward()
        #def closure():
        #    return b_energyeig(x1,n,nb,Mass,Charge,Sym,symc)
        #optimizer.step(closure)
        optimizer.step()
        #scheduler.step(loss)
        
        print('step: {} f: {} gradNorm: {}'.format(i, loss, th.norm(x1.grad)))
    
    return x1

In [219]:
th.set_printoptions(linewidth=120)
start_time = time.time()
xrestart = opt_energyrc()
print(" took {} seconds ".format(time.time() - start_time))
#cProfile.run('xprof = opt_energyrc()')

step: 0 f: -7.4779189029342135 gradNorm: 7.834516230206545e-05
step: 1 f: -7.4779186933917545 gradNorm: 0.0008307394094858418
step: 2 f: -7.477918757059968 gradNorm: 0.0005681336481752616
step: 3 f: -7.477918857296963 gradNorm: 0.0004009212136136211
step: 4 f: -7.477918903264577 gradNorm: 0.00035051358005092694
step: 5 f: -7.477918954819422 gradNorm: 0.00022554963322933926
step: 6 f: -7.4779189738866165 gradNorm: 0.0002587389068854611
step: 7 f: -7.477918989507535 gradNorm: 0.00024638097962919844
step: 8 f: -7.477919014489875 gradNorm: 0.00012901437452968626
step: 9 f: -7.477919033156086 gradNorm: 0.00018695998768494232
step: 10 f: -7.477919056370425 gradNorm: 0.00013083436532287953
step: 11 f: -7.477919080456053 gradNorm: 9.445104287462148e-05
step: 12 f: -7.477919104724303 gradNorm: 9.353410753321703e-05
step: 13 f: -7.4779191322842085 gradNorm: 4.987125340723182e-05
step: 14 f: -7.4779191583474445 gradNorm: 9.004322136161116e-05
step: 15 f: -7.477919189050102 gradNorm: 6.32423384820

step: 130 f: -7.4779229898170865 gradNorm: 7.751101412152724e-05
step: 131 f: -7.477923024749926 gradNorm: 8.937150138005065e-05
step: 132 f: -7.477923058411677 gradNorm: 9.994459319986703e-05
step: 133 f: -7.477923096877504 gradNorm: 8.044474508868353e-05
step: 134 f: -7.477923136018599 gradNorm: 0.00010805134871720293
step: 135 f: -7.477923172124355 gradNorm: 9.219273854051081e-05
step: 136 f: -7.477923202585637 gradNorm: 0.00010655009276050486
step: 137 f: -7.477923231161258 gradNorm: 0.00011620609150994932
step: 138 f: -7.47792325673379 gradNorm: 8.938118791060644e-05
step: 139 f: -7.477923283475562 gradNorm: 7.041912541430324e-05
step: 140 f: -7.477923305805631 gradNorm: 8.223385692953602e-05
step: 141 f: -7.477923330335146 gradNorm: 0.00011990147542187204
step: 142 f: -7.477923356603179 gradNorm: 0.000113595971815075
step: 143 f: -7.47792337827688 gradNorm: 8.090088449157681e-05
step: 144 f: -7.477923404388026 gradNorm: 8.161977527941646e-05
step: 145 f: -7.477923430026165 gradNo

step: 258 f: -7.477926651232881 gradNorm: 8.356260953077371e-05
step: 259 f: -7.477926675378738 gradNorm: 7.02558819773988e-05
step: 260 f: -7.477926700662408 gradNorm: 7.281476384174753e-05
step: 261 f: -7.47792672355786 gradNorm: 6.590265336072969e-05
step: 262 f: -7.477926749455479 gradNorm: 6.313110599103305e-05
step: 263 f: -7.477926776638213 gradNorm: 6.558351825492736e-05
step: 264 f: -7.477926804566931 gradNorm: 6.897985551546276e-05
step: 265 f: -7.477926831291599 gradNorm: 8.20395134045531e-05
step: 266 f: -7.477926859609785 gradNorm: 7.819392951117597e-05
step: 267 f: -7.477926885067533 gradNorm: 7.106099181210468e-05
step: 268 f: -7.477926910382352 gradNorm: 6.402982800912366e-05
step: 269 f: -7.477926935198309 gradNorm: 6.640542240368902e-05
step: 270 f: -7.477926963456437 gradNorm: 6.204545704992168e-05
step: 271 f: -7.477926993098561 gradNorm: 7.334234551578293e-05
step: 272 f: -7.4779270233057575 gradNorm: 8.254832512447573e-05
step: 273 f: -7.477927055858047 gradNorm: 

step: 386 f: -7.477930034858622 gradNorm: 6.274150545954518e-05
step: 387 f: -7.477930057361114 gradNorm: 5.999633389274948e-05
step: 388 f: -7.4779300832943365 gradNorm: 5.516773227393528e-05
step: 389 f: -7.477930109770845 gradNorm: 6.450661540499508e-05
step: 390 f: -7.477930138091271 gradNorm: 7.172519666583358e-05
step: 391 f: -7.477930167261871 gradNorm: 6.883145628712547e-05
step: 392 f: -7.477930194865147 gradNorm: 6.598854530171878e-05
step: 393 f: -7.4779302207980685 gradNorm: 7.225360797466329e-05
step: 394 f: -7.477930242574896 gradNorm: 0.00011158779174090584
step: 395 f: -7.477930265458145 gradNorm: 8.202976682260224e-05
step: 396 f: -7.477930287917931 gradNorm: 6.060987773451948e-05
step: 397 f: -7.477930309142793 gradNorm: 6.764078773601232e-05
step: 398 f: -7.477930329660105 gradNorm: 6.975294497893555e-05
step: 399 f: -7.477930350709192 gradNorm: 6.827561878602881e-05
step: 400 f: -7.477930374702285 gradNorm: 6.702737883361973e-05
step: 401 f: -7.477930399111149 gradN

step: 515 f: -7.477933146997643 gradNorm: 8.140688000588188e-05
step: 516 f: -7.477933166426658 gradNorm: 6.738624089648593e-05
step: 517 f: -7.477933185899466 gradNorm: 7.784716826973048e-05
step: 518 f: -7.477933208036249 gradNorm: 7.923982786105749e-05
step: 519 f: -7.4779332325514085 gradNorm: 7.772526736884174e-05
step: 520 f: -7.477933260792691 gradNorm: 7.543376648416065e-05
step: 521 f: -7.477933292242463 gradNorm: 7.733318562596086e-05
step: 522 f: -7.477933324439675 gradNorm: 6.25477796681951e-05
step: 523 f: -7.4779333451732395 gradNorm: 8.356370480963142e-05
step: 524 f: -7.4779333583715575 gradNorm: 0.0001276131436082465
step: 525 f: -7.477933392691458 gradNorm: 7.574982180585024e-05
step: 526 f: -7.477933414962163 gradNorm: 0.00011257130836399188
step: 527 f: -7.4779334306894345 gradNorm: 0.0001107814970255744
step: 528 f: -7.4779334508172255 gradNorm: 0.00010078043750909499
step: 529 f: -7.477933472693908 gradNorm: 0.00010098605781746036
step: 530 f: -7.477933495791592 g

...

step: 1406 f: -7.477949022518427 gradNorm: 9.236328117660087e-05
step: 1407 f: -7.477949036999834 gradNorm: 7.755252143270362e-05
step: 1408 f: -7.477949051397437 gradNorm: 6.222637579847491e-05
step: 1409 f: -7.477949065854511 gradNorm: 6.577326418977219e-05
step: 1410 f: -7.477949079295499 gradNorm: 7.71979164183047e-05
step: 1411 f: -7.477949093805693 gradNorm: 6.371065362139796e-05
step: 1412 f: -7.47794910917258 gradNorm: 7.072000083162065e-05
step: 1413 f: -7.477949123572721 gradNorm: 6.213355042183145e-05
step: 1414 f: -7.4779491386953 gradNorm: 6.205505446892239e-05
step: 1415 f: -7.47794915157844 gradNorm: 6.23598823969965e-05
step: 1416 f: -7.477949165400629 gradNorm: 6.79609499139643e-05
step: 1417 f: -7.477949179821632 gradNorm: 5.326474217088237e-05
step: 1418 f: -7.477949191617502 gradNorm: 0.00010579525260800197
step: 1419 f: -7.4779492066994715 gradNorm: 6.791759983069932e-05
step: 1420 f: -7.47794921894269 gradNorm: 9.213600386247954e-05
step: 1421 f: -7.47794923336217

step: 1532 f: -7.477950801925997 gradNorm: 5.523075515210544e-05
step: 1533 f: -7.477950815413787 gradNorm: 6.600930609371557e-05
step: 1534 f: -7.477950830101466 gradNorm: 5.5906176492642525e-05
step: 1535 f: -7.477950845064776 gradNorm: 5.2074746619774574e-05
step: 1536 f: -7.4779508607482015 gradNorm: 6.358387452158813e-05
step: 1537 f: -7.477950877156487 gradNorm: 6.264847958943345e-05
step: 1538 f: -7.477950894398203 gradNorm: 5.698693861322502e-05
step: 1539 f: -7.4779509116096525 gradNorm: 6.341097581354143e-05
step: 1540 f: -7.477950925076187 gradNorm: 7.683587273172165e-05
step: 1541 f: -7.477950940761342 gradNorm: 7.031577126470403e-05
step: 1542 f: -7.477950952163669 gradNorm: 6.816627772172702e-05
step: 1543 f: -7.477950964770753 gradNorm: 6.394890248140205e-05
step: 1544 f: -7.477950977526014 gradNorm: 6.527480842409016e-05
step: 1545 f: -7.477950990408307 gradNorm: 5.237853239555485e-05
step: 1546 f: -7.477951004225822 gradNorm: 5.259803082551402e-05
step: 1547 f: -7.4779

step: 1658 f: -7.4779524525293075 gradNorm: 4.859660548826677e-05
step: 1659 f: -7.477952466709837 gradNorm: 4.998363063270247e-05
step: 1660 f: -7.477952481995783 gradNorm: 4.996957316893929e-05
step: 1661 f: -7.477952498119032 gradNorm: 6.1743681220041e-05
step: 1662 f: -7.477952515103155 gradNorm: 5.19604248794348e-05
step: 1663 f: -7.477952531669227 gradNorm: 6.5916756829673e-05
step: 1664 f: -7.477952543208358 gradNorm: 8.103349272458435e-05
step: 1665 f: -7.477952561315411 gradNorm: 5.529228280094978e-05
step: 1666 f: -7.477952574597949 gradNorm: 5.6967165363698e-05
step: 1667 f: -7.477952588553035 gradNorm: 6.170575212938718e-05
step: 1668 f: -7.477952603397976 gradNorm: 4.464855093888615e-05
step: 1669 f: -7.477952617519436 gradNorm: 5.55813482602252e-05
step: 1670 f: -7.477952632755703 gradNorm: 5.3304687433838725e-05
step: 1671 f: -7.477952648219475 gradNorm: 5.213952363627233e-05
step: 1672 f: -7.477952663156637 gradNorm: 5.8851543372321573e-05
step: 1673 f: -7.4779526788193

step: 1784 f: -7.477954485321945 gradNorm: 5.2340788541609617e-05
step: 1785 f: -7.4779545044093805 gradNorm: 4.920845612642713e-05
step: 1786 f: -7.47795452039844 gradNorm: 5.61994116246406e-05
step: 1787 f: -7.4779545390525195 gradNorm: 5.973166094074947e-05
step: 1788 f: -7.47795455830671 gradNorm: 5.7622985678021407e-05
step: 1789 f: -7.4779545785511425 gradNorm: 5.584934870351489e-05
step: 1790 f: -7.477954596284619 gradNorm: 7.222697253282865e-05
step: 1791 f: -7.477954617022043 gradNorm: 4.941478828294198e-05
step: 1792 f: -7.477954638866892 gradNorm: 5.150848024731463e-05
step: 1793 f: -7.477954661652847 gradNorm: 6.872486024700726e-05
step: 1794 f: -7.477954687179526 gradNorm: 5.401426295010808e-05
step: 1795 f: -7.477954712605888 gradNorm: 6.271946437278657e-05
step: 1796 f: -7.47795473608487 gradNorm: 7.068672158818317e-05
step: 1797 f: -7.4779547586101005 gradNorm: 7.03677478106362e-05
step: 1798 f: -7.47795478429894 gradNorm: 6.058266995147158e-05
step: 1799 f: -7.47795480

step: 1910 f: -7.4779571712994715 gradNorm: 6.31266239305758e-05
step: 1911 f: -7.477957188677246 gradNorm: 4.7950066755396546e-05
step: 1912 f: -7.477957202468926 gradNorm: 7.68250021927125e-05
step: 1913 f: -7.477957218028331 gradNorm: 6.738295788982521e-05
step: 1914 f: -7.477957231373108 gradNorm: 6.101584230823506e-05
step: 1915 f: -7.477957246486039 gradNorm: 5.0944546136043695e-05
step: 1916 f: -7.477957256785279 gradNorm: 5.203612685370814e-05
step: 1917 f: -7.47795727011976 gradNorm: 4.1758499821354155e-05
step: 1918 f: -7.477957286619485 gradNorm: 4.424116087662363e-05
step: 1919 f: -7.47795729898337 gradNorm: 5.235086614360119e-05
step: 1920 f: -7.477957311240351 gradNorm: 5.877404515439243e-05
step: 1921 f: -7.477957323544824 gradNorm: 4.518834208739473e-05
step: 1922 f: -7.477957336001364 gradNorm: 5.019236674103897e-05
step: 1923 f: -7.4779573487550985 gradNorm: 3.979467351612212e-05
step: 1924 f: -7.477957362531556 gradNorm: 4.509703687209123e-05
step: 1925 f: -7.4779573

In [220]:
th.save(xrestart, 'Libo-nb128-7.4778.pt')

In [212]:
xrestart = th.load('Libo-nb128-7.4778.pt')